## Toy MHD and divergence cleaning 


We made a code to check the divergence cleaning strategies discussed in: https://www.overleaf.com/read/phmtkqtkxwjr#baf932

In this version we add the hyperbolic divergence cleaning. Since we are using periodic boundary conditions we don't expect much improvement, only the one coming from the damping term. 

In [ ]:
using Plots
using SummationByPartsOperators
using LinearAlgebra
using ArraysOfArrays
using Base.Threads
using Revise
using JLD2

includet("../PIC/PIC-1D/aux_functions/aux_functions_grid.jl")
includet("../PIC/PIC-1D/aux_functions/aux_functions_E-B.jl")
includet("../PIC/PIC-1D/aux_functions/aux_functions_RHS.jl")
includet("toy_MHD_auxfunctions.jl")

In [ ]:
J = (100,100)
Box = (0.0,1.0,0.0,1.0)
dx = differentials(Box,J)
const D = 2

xv = [(i-1)*dx[1] for i in 1:J[1]]
yv = [(i-1)*dx[2] for i in 1:J[2]];

In [ ]:
Dx = periodic_derivative_operator(derivative_order=1, accuracy_order=4, xmin=Box[1], xmax=Box[2], N=J[1])
Dy = periodic_derivative_operator(derivative_order=1, accuracy_order=4, xmin=Box[3], xmax=Box[4], N=J[2])
Δx = dissipation_operator(Dx;
                     #mode=D.coefficients.mode
                     #,mode=ThreadedMode()
                     )
Δy = dissipation_operator(Dy;
                     #mode=D.coefficients.mode
                     #,mode=ThreadedMode()
                     )
σx = 0.0 #dissipation strength
σy = 0.0 #dissipation strength

Build initial data and velocity field.

In [ ]:
k1 = 2π*[1,1]
k2 = 2π*[2,2]
x0 = [0.5,0.5]
fv(x,y,k) = sin(k[1]*x)*sin(k[2]*y)
function fp(x,y,x0,p) 
    r20 = 0.02
    r2 = (x-x0[1])^2+(y-x0[2])^2
    if r2 - r20 < 0
        return (r2 - r20)^p/r20^p
    else
        return 0.0
    end
end
V = [-(l-2)*sin(k1[1]*xv[i]) + (l-1)*sin(k1[2]*yv[j]) for l in 1:2, i in 1:J[1], j in 1:J[2]];
B = [(l-2)*k2[2]*sin(k2[1]*xv[i])*cos(k2[2]*yv[j]) + (l-1)*k2[1]*cos(k2[1]*xv[i])*sin(k2[2]*yv[j]) for l in 1:2, i in 1:J[1], j in 1:J[2]];


In [ ]:
surface(yv,xv,V[1,:,:])

In [ ]:
surface(yv,xv,B[2,:,:])

In [ ]:
@show maximum(abs.(Div(B,Dx,Dy,J)))

In [ ]:
surface(yv,xv,Div(B,Dx,Dy,J))

In [ ]:
surface(yv,xv,Div(V,Dx,Dy,J))

In [ ]:
u = Vector{Float64}(undef,(D+1)*J[1]*J[2]); #the last component is psi
du = Vector{Float64}(undef,(D+1)*J[1]*J[2]);

u .= 0.0;
du .= 0.0;

U = reshape(u,D+1,J[1],J[2]);



#F = B


The evolution equations are:

$$
\partial_t B^i = \nabla_j (v^j B^i - v^i B^j) + \kappa \nabla^i (\nabla_j B^j)
$$
for the parabolic case, and

\begin{align*}
\partial_t B^i &= \nabla_j (v^j B^i - v^i B^j) + \nabla^i ψ \\
\partial_t ψ &= c^2 \nabla_j B^j - τψ
\end{align*}


In [ ]:
U[1:D,:,:] .= copy(B)
U[D+1,:,:] .= 0.0 #psi
@show B === reshape(u,(D+1,J...))[1:D,:,:]
@show norm(B - reshape(u,(D+1,J...))[1:D,:,:])
@show maximum(abs.(u))
@show get_Energy(u,J,Box)
@show maximum(abs.(Div(reshape(u,(D+1,J...))[1:D,:,:],Dx,Dy,J)))

@show typeof(u)
@show typeof(F_hyp!(u,0.0,(J, Dx, Δx, Dy, Δy, σx, σy, V, 0.1, 1.0, 1.0, du)))

surface(yv,xv,reshape(F_hyp!(u,0.0,(J, Dx, Δx, Dy, Δy, σx, σy, zeros(D,J...), 0.1, 1, 1, du)),D+1,J...)[3,:,:])
#surface(yv,xv,du[2,:,:])

In [ ]:
(t_i, t_f) = (0.0, 200.0)
M = 40000
@show dt = (t_f - t_i)/M
@show differentials(Box,J)
κ = maximum(dx)
#κ = 0.0
@show κ 
par = (J, Dx, Δx, Dy, Δy, σx, σy, V, κ, 0.0, 0.0, du)
      #(J, Dx, Δx, Dy, Δy, σx, σy, V, 0.1, du)
t = t_i
Energy = Array{Float64,1}(undef,M+1)
Div_B_max = Array{Float64,1}(undef,M+1)
Div_B_L2 = Array{Float64,1}(undef,M+1)
tt = Array{Float64,1}(undef,M+1)
@show Energy[1] = get_Energy(u,J,Box)
@show Div_B_max[1] = maximum(abs.(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J)))
@show Div_B_L2[1] = norm(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J))*sqrt(volume(Box)/prod(J))
tt[1] = t

In [ ]:
for i in 1:M
    RK4_Step!(F_hyp!, u, 0.0, dt, par)
    t = t + dt
    tt[i+1] = t
    
    Energy[i+1] = get_Energy(u,J,Box)
    Div_B_max[i+1] = maximum(abs.(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J)))
    Div_B_L2[i+1] = norm(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J))*sqrt(volume(Box)/prod(J))

    println("t = $t")
    println("Energy = $(get_Energy(u,J,Box))")
    println("DivB = $(maximum(abs.(Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J))))")
    
end

In [ ]:
surface(yv,xv,Div(reshape(u,(D+1,J...))[1:D, :, :],Dx,Dy,J))

In [ ]:
plot(tt, Energy, yscale=:log10, title="Energy vs time", xlabel="t", ylabel="Energy")

In [ ]:
plot(tt, Div_B_max,title="Max |div B| vs time",xlabel="t",ylabel="Max |div B|")

In [ ]:
plot(tt, Div_B_L2,title="Norm |div B| vs time",xlabel="t",ylabel="Max |div B|")

In [ ]:
Div_B_max_0 = copy(Div_B_max);
#Div_B_max_1_1 = copy(Div_B_max);
Div_B_L2_0 = copy(Div_B_L2);
#Div_B_L2_1_1 = copy(Div_B_L2);


In [ ]:
plot(tt,Div_B_L2_0, title="Norm |div B| vs time D-order=4",xlabel="t",ylabel="Norm |div B|", label="κ=0.0", yscale=:log10)
plot!(tt,Div_B_L2_1_1, label="c2=1 τ=1")
#savefig("DivB_L2_vs_time_κD-order=4.png")

![DivB_L2_vs_time](DivB_L2_vs_time_κ.png)

In [ ]:
plot(tt,Div_B_max_0, title="Max |div B| vs time D-order=4",xlabel="t",ylabel="Max |div B|", label="c2=0.0", yscale=:log10)
plot!(tt,Div_B_max_1_1, label="c2=1 τ=1")
#savefig("DivB_max_vs_time_κ_D-order=4.png")

![DivB_max_vs_time](DivB_max_vs_time_κ.png)